In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [4]:
base_path = r"C:\Users\이수진\GitHub\SERVER_WAYVI\ai-server\datasets\raw\BITS-2 Fall Detection Dataset"
FIXED_LENGTH = 5  # 시계열 길이
NUM_FEATURES = 7   # acc(3) + gyro(3) + hr(1)
LABEL_MAP = {"adl": 0, "fall": 1}

In [5]:
def process_sensor_file(file_path):
    df = pd.read_csv(file_path, header=None)
    
    acc_rows, gyro_rows, hrt_rows = [], [], []

    for _, row in df.iterrows():
        sensor = None

        if row.iloc[5] in ['acc', 'gyro']:
            sensor = row.iloc[5]
        elif row.iloc[3] == 'hrt':
            sensor = 'hrt'
        else:
            continue 

        if sensor == 'acc':
            acc_rows.append(row.iloc[:5].tolist())
        elif sensor == 'gyro':
            gyro_rows.append(row.iloc[:5].tolist())
        elif sensor == 'hrt':
            hrt_rows.append(row.iloc[:3].tolist())

    dfs = []

    if acc_rows:
        acc_df = pd.DataFrame(acc_rows, columns=['t', 'acc_x', 'acc_y', 'acc_z', 'acc_a'])
        acc_df = acc_df[['t', 'acc_x', 'acc_y', 'acc_z']].apply(pd.to_numeric, errors='coerce')
        acc_df = acc_df.dropna(subset=["t"]).groupby('t').mean().reset_index()
        dfs.append(acc_df)

    if gyro_rows:
        gyro_df = pd.DataFrame(gyro_rows, columns=['t', 'gyro_x', 'gyro_y', 'gyro_z', 'gyro_a'])
        gyro_df = gyro_df[['t', 'gyro_x', 'gyro_y', 'gyro_z']].apply(pd.to_numeric, errors='coerce')
        gyro_df = gyro_df.dropna(subset=["t"]).groupby('t').mean().reset_index()
        dfs.append(gyro_df)
        
    if hrt_rows:
        hrt_df = pd.DataFrame(hrt_rows, columns=['t', 'bpm', 'a'])
        hrt_df['t'] = pd.to_numeric(hrt_df['t'], errors='coerce')
        hrt_df['bpm'] = pd.to_numeric(hrt_df['bpm'], errors='coerce')
        hrt_df = hrt_df.groupby('t')[['bpm']].mean().reset_index()
        dfs.append(hrt_df)

    merged_df = dfs[0]
    for sub_df in dfs[1:]:
        merged_df = pd.merge(merged_df, sub_df, on='t', how='outer')

    merged_df = merged_df.dropna().reset_index(drop=True)

    return merged_df

In [7]:
merged_df = process_sensor_file(r"C:\Users\이수진\GitHub\SERVER_WAYVI\ai-server\datasets\raw\BITS-2 Fall Detection Dataset/adl/user1/user1_adl1.csv")

In [21]:
merged_df

,t,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,bpm
0,1.830000e+11,-0.370303,0.059057,-1.268927e-01,-0.055792,0.015882,0.149458,100.0
1,1.840000e+11,0.105003,0.036597,-1.764869e-01,0.214326,0.004887,-0.164585,100.0
2,1.850000e+11,0.045490,-0.011971,1.975216e-01,-0.096313,-0.040317,0.133983,100.0
3,1.860000e+11,0.167195,-0.028331,8.379695e-03,0.256360,0.084503,-0.176540,104.0
4,1.870000e+11,-0.138066,-0.025937,8.020576e-02,-0.136834,-0.031561,0.106291,106.0
5,1.880000e+11,0.081802,-0.023543,-1.145226e-01,0.086539,-0.002036,-0.143350,107.0
6,1.890000e+11,0.023600,-0.160069,1.826434e-01,-0.107338,0.048171,0.106989,108.0
7,1.900000e+11,-0.021548,-0.122503,2.114878e-01,0.041132,-0.024638,-0.204233,106.0
8,1.910000e+11,0.481235,0.061451,-9.776324e-02,0.057218,-0.076765,0.129707,105.0
9,1.920000e+11,-0.262165,-0.304064,-9.457096e-02,-0.221337,0.131743,-0.082060,104.0


In [12]:
X = []
y = []

for label_name in ['adl', 'fall']:
    label_dir = os.path.join(base_path, label_name)
    label_value = 0 if label_name == 'adl' else 1

    for user_folder in os.listdir(label_dir):
        user_path = os.path.join(label_dir, user_folder)
        if not os.path.isdir(user_path):
            continue

        for file in os.listdir(user_path):
            if not file.endswith(".csv"):
                continue

            file_path = os.path.join(user_path, file)
            df = process_sensor_file(file_path)
            if df is None or df.empty:
                continue

            data = df.drop(columns=["t"]).to_numpy()

            # NaN이 포함된 행 제거
            if np.isnan(data).any():
                continue

            total_len = data.shape[0]
            num_chunks = total_len // FIXED_LENGTH

            for i in range(num_chunks):
                chunk = data[i * FIXED_LENGTH : (i + 1) * FIXED_LENGTH]
                if chunk.shape[0] == FIXED_LENGTH:
                    X.append(chunk)
                    y.append(label_value)

In [14]:
X

[array([[-3.70303192e-01,  5.90569767e-02, -1.26892693e-01,
         -5.57923592e-02,  1.58824933e-02,  1.49458360e-01,
          1.00000000e+02],
        [ 1.05002856e-01,  3.65970893e-02, -1.76486861e-01,
          2.14326434e-01,  4.88692286e-03, -1.64584549e-01,
          1.00000000e+02],
        [ 4.54898273e-02, -1.19710117e-02,  1.97521644e-01,
         -9.63130900e-02, -4.03171067e-02,  1.33983114e-01,
          1.00000000e+02],
        [ 1.67195083e-01, -2.83313817e-02,  8.37969500e-03,
          2.56359781e-01,  8.45030268e-02, -1.76540056e-01,
          1.04000000e+02],
        [-1.38065631e-01, -2.59371817e-02,  8.02057587e-02,
         -1.36833811e-01, -3.15613702e-02,  1.06290553e-01,
          1.06000000e+02]]),
 array([[ 8.18018923e-02, -2.35429850e-02, -1.14522645e-01,
          8.65392458e-02, -2.03621500e-03, -1.43349712e-01,
          1.07000000e+02],
        [ 2.35999853e-02, -1.60069491e-01,  1.82643386e-01,
         -1.07337739e-01,  4.81710833e-02,  1.06988686e-

In [18]:
X = np.stack(X)  # (N, T, C)
y = np.array(y)  # (N,)

In [20]:
Ｘ

array([[[-3.70303192e-01,  5.90569767e-02, -1.26892693e-01, ...,
          1.58824933e-02,  1.49458360e-01,  1.00000000e+02],
        [ 1.05002856e-01,  3.65970893e-02, -1.76486861e-01, ...,
          4.88692286e-03, -1.64584549e-01,  1.00000000e+02],
        [ 4.54898273e-02, -1.19710117e-02,  1.97521644e-01, ...,
         -4.03171067e-02,  1.33983114e-01,  1.00000000e+02],
        [ 1.67195083e-01, -2.83313817e-02,  8.37969500e-03, ...,
          8.45030268e-02, -1.76540056e-01,  1.04000000e+02],
        [-1.38065631e-01, -2.59371817e-02,  8.02057587e-02, ...,
         -3.15613702e-02,  1.06290553e-01,  1.06000000e+02]],

       [[ 8.18018923e-02, -2.35429850e-02, -1.14522645e-01, ...,
         -2.03621500e-03, -1.43349712e-01,  1.07000000e+02],
        [ 2.35999853e-02, -1.60069491e-01,  1.82643386e-01, ...,
          4.81710833e-02,  1.06988686e-01,  1.08000000e+02],
        [-2.15478027e-02, -1.22503318e-01,  2.11487796e-01, ...,
         -2.46382308e-02, -2.04232610e-01,  1.06000

In [25]:
print("X shape:", X.shape)  
print("y shape:", y.shape)  

X shape: (57, 5, 7)
y shape: (57,)


In [27]:
print("X NaN:", np.isnan(X).sum())
print("X Inf:", np.isinf(X).sum())
print("y NaN:", np.isnan(y).sum())
print("y Inf:", np.isinf(y).sum())

X NaN: 0
X Inf: 0
y NaN: 0
y Inf: 0


In [29]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1])

In [243]:
def normalize_dataset(X):
    N, T, C = X.shape
    X_reshaped = X.reshape(-1, C)  # (N*T, C)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_reshaped)
    return X_scaled.reshape(N, T, C), scaler

In [245]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train, scaler = normalize_dataset(X_train)
X_test = scaler.transform(X_test.reshape(-1, X.shape[2])).reshape(X_test.shape)

In [247]:
# 저장 경로 생성
save_dir = "./processed_data"
os.makedirs(save_dir, exist_ok=True)

# 저장
np.save(os.path.join(save_dir, "X_train.npy"), X_train)
np.save(os.path.join(save_dir, "X_test.npy"), X_test)
np.save(os.path.join(save_dir, "y_train.npy"), y_train)
np.save(os.path.join(save_dir, "y_test.npy"), y_test)

In [267]:
class FallDetectionModel(nn.Module):
    def __init__(self, input_dim=17, seq_len=50, hidden_dim=64):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
        )
        self.lstm = nn.LSTM(input_size=64, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # ✅ sigmoid 제거!

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.cnn(x)
        x = x.transpose(1, 2)
        _, (h_n, _) = self.lstm(x)
        out = self.fc(h_n[-1])
        return out.squeeze(1)  # BCEWithLogitsLoss expects raw logits

In [251]:
class EarlyStopping:
    def __init__(self, patience=5, delta=1e-4):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = np.inf
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [253]:
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.float32)  # BCEWithLogitsLoss expects float

# Dataset & Dataloader
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64)

In [269]:
model = FallDetectionModel(input_dim=X_train.shape[2]).to("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

device = next(model.parameters()).device

early_stopper = EarlyStopping(patience=5)

for epoch in range(50):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_losses = []
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            val_losses.append(loss.item())
    print(f"Epoch {epoch+1}, Val Loss: {np.mean(val_losses):.4f}")

Epoch 1, Val Loss: 0.6262
Epoch 2, Val Loss: 0.5968
Epoch 3, Val Loss: 0.5497
Epoch 4, Val Loss: 0.5154
Epoch 5, Val Loss: 0.5158
Epoch 6, Val Loss: 0.5049
Epoch 7, Val Loss: 0.5621
Epoch 8, Val Loss: 0.5393
Epoch 9, Val Loss: 0.5517
Epoch 10, Val Loss: 0.7106
Epoch 11, Val Loss: 0.5889
Epoch 12, Val Loss: 0.5753
Epoch 13, Val Loss: 0.5822
Epoch 14, Val Loss: 0.6124
Epoch 15, Val Loss: 0.5451
Epoch 16, Val Loss: 0.6407
Epoch 17, Val Loss: 0.6463
Epoch 18, Val Loss: 0.4903
Epoch 19, Val Loss: 0.6295
Epoch 20, Val Loss: 0.6867
Epoch 21, Val Loss: 0.5707
Epoch 22, Val Loss: 0.6474
Epoch 23, Val Loss: 0.6777
Epoch 24, Val Loss: 0.7144
Epoch 25, Val Loss: 0.6300
Epoch 26, Val Loss: 0.7177
Epoch 27, Val Loss: 0.7877
Epoch 28, Val Loss: 0.8847
Epoch 29, Val Loss: 0.5808
Epoch 30, Val Loss: 0.6057
Epoch 31, Val Loss: 0.6962
Epoch 32, Val Loss: 0.6448
Epoch 33, Val Loss: 0.8373
Epoch 34, Val Loss: 0.7651
Epoch 35, Val Loss: 0.6822
Epoch 36, Val Loss: 0.7003
Epoch 37, Val Loss: 0.7191
Epoch 38, 

In [ ]:
probs = torch.sigmoid(pred)
pred_labels = (probs > 0.5).int()

In [273]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 1. 데이터 준비
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# 2. 모델 추론
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    probs = torch.sigmoid(outputs)
    preds = (probs > 0.5).cpu().numpy()  # 이진 예측
    targets = y_test_tensor.numpy()

# 3. 성능 지표 계산
acc = accuracy_score(targets, preds)
prec = precision_score(targets, preds, zero_division=0)
rec = recall_score(targets, preds, zero_division=0)
f1 = f1_score(targets, preds, zero_division=0)
cm = confusion_matrix(targets, preds)

# 4. 출력
print("Confusion Matrix:")
print(cm)
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")


Confusion Matrix:
[[112  19]
 [ 17  49]]
Accuracy : 0.8173
Precision: 0.7206
Recall   : 0.7424
F1-score : 0.7313


In [275]:
torch.save(model.state_dict(), "fall_detection_model.pt")